In [1]:
class Subset:
    
    def __init__(self, subset, weight, ID):
        self.subset = subset
        self.weight = weight
        self.ID = ID
        
    def get_subset(self):
        return self.subset
    
    def get_weight(self):
        return self.weight
    
    def get_ID(self):
        return self.ID
    
    def contains(self, n):
        contains = False
        for m in subset:
            if m == n:
                contains = True
        return contains

    def get_size(self):
        return len(self.subset)
    
    def to_string(self):
        output = ""
        for n in self.subset:
            output = output+str(n)+" "
        output = output + "\n"+str(self.weight)
        return output

In [15]:
class ALGOBowl:
    
    def __init__(self):
        self.n=0
        self.m=0
        self.subsets = []
        self.solution = []

    def read_input(self, filename):
        file = open(filename,'r')
        self.n = int(file.readline())
        self.m = int(file.readline())
        for i in range(1,self.m+1):
            temp = []
            for string in file.readline().split(" "):
                if(string == "\n"):
                    continue
                else:
                    temp.append(int(string))
            weight = int(file.readline())
            ss = Subset(temp,weight,i)
            self.subsets.append(ss)
    
    def display_data(self):
        print(self.n)
        print(self.m)
        for s in self.subsets:
            print(s.to_string())

# not sure if usefull
#     def get_subsets():
#         sets = []
#         for s in subsets:
#             sets.append(s.get_subset())
#         return sets
    
    def disjoint(self, *args, sets = None):
        if sets == None:
            nums = set()
            for s in args:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            return True
        else:
            nums = set()
            for s in sets:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            for s in args:
                for n in s:
                    if n in nums:
                        return False
                    else:
                        nums.add(n)
            return True
   
    def run_most_disjoint(self):
        del self.solution[:]
        IDs = []
        solution = []
        total_weight=0
        nums = set()
        for s in self.subsets:
            if self.disjoint(s.get_subset(), sets = solution):
                solution.append(s.get_subset())
                IDs.append(s.get_ID())
                total_weight = total_weight + s.get_weight()
                nums.update(s.get_subset())
        if len(nums) != self.n:
            rest = self.find_needed_subsets(nums)
            for ids in rest[0]:
                IDs.append(ids)
            total_weight = total_weight + rest[1]
            for s in rest[2]:
                solution.append(s)
        IDs.sort()
        self.solution = [IDs, total_weight, solution]
        return [IDs, total_weight, solution]

#might be working now
    def find_needed_subsets(self, nums):#helper function for disjoint algorithm^^^^
        IDs = []
        solution = []
        total_weight =0
        needed_numbers = set()
        for n in range(1,self.n+1):
            if not n in nums:
                needed_numbers.add(n)
        for s in self.subsets:
            for n in s.get_subset():
                if n in nums:
                    continue
                if n in needed_numbers:
                    IDs.append(s.get_ID())
                    solution.append(s.get_subset())
                    total_weight = total_weight + s.get_weight()
                    nums.add(n)
                    break

#         for n in needed_numbers:
#             if n in nums:
#                 continue
#             for s in self.subsets:
#                 for m in s.get_subset():
#                     if n == m:
#                         IDs.append(s.get_ID())
#                         solution.append(s.get_subset())
#                         total_weight = total_weight + s.get_weight()
#                         nums.add(n)
#                         break
        return [IDs, total_weight, solution]
                               
        
    def print_solution(self):
        output = str(self.solution[1])+"\n"
        for s in self.solution[0]:
            output = output + str(s) + " "
        return output
    
    def write_output(self, filename):
        file = open(filename, "w")
        file.write(self.print_solution())
        file.flush()
        file.close()
    
            

In [18]:
#main
index = 2
inputs = ["easy", "input_10", "input_50", "input_100", "input_500"]
ext = ".txt"
output = "_output.txt"
test = ALGOBowl()
test.read_input(inputs[index]+ext)
# test.display_data()
test.run_most_disjoint()
print(test.print_solution())
test.write_output(inputs[index]+output)

323
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
